In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from google.protobuf import text_format
from ffn.inference import inference
from ffn.inference import inference_pb2
from ffn.inference import storage
from ffn.inference import seed as sd


import numpy as np
import yaml
import zipfile
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [3]:
current = '104-001'
trial = '7c'
checkpoint = '22000000'
INTERACTIVE = True
EXTRACT_ZIPS = True

In [4]:
path_to_yaml = '/app/julien/retinaFFN/inference/inference_config.yaml'
stream = open(path_to_yaml,'r')
images = yaml.load(stream, Loader=yaml.FullLoader)
params = images[current]
seed = params['seed']
corner = params['corner']
subvol_size = params['subvol_size']

In [5]:
config = '''image {
  hdf5: "/app/julien/validation/sumbul/single_image/Image''' + current + '''_01/bbox/Image''' + current + '''_01_bbox_zyx.hdf5:bbox_zyx"
}
image_mean: 128
image_stddev: 33
checkpoint_interval: 5
seed_policy: "PolicyManual"
seed_policy_args: "{\\"points\\": [''' + str(seed[0]-corner[0]) + ''',''' + str(seed[1]-corner[1]) + ''',''' + str(seed[2]-corner[2]) + ''']}"
model_checkpoint_path: "/app/julien/models/trial''' + trial + '''/model.ckpt-''' + checkpoint + '''"
model_name: "convstack_3d.ConvStack3DFFNModel"
model_args: "{\\"depth\\": 5, \\"fov_size\\": [12, 12, 12], \\"deltas\\": [2, 2, 2]}"
segmentation_output_dir: "/app/julien/validation/sumbul/single_image/Image''' + current + '''_01/segmentation_trial'''+ trial +'''"
inference_options {
  init_activation: 0.95
  pad_value: 0.05
  move_threshold: 0.7
  min_boundary_dist { x: 1 y: 1 z: 1}
  segment_threshold: 0.5
  min_segment_size: 1000
  disco_seed_threshold: 0.1
  consistency_threshold: 0.1
}'''

In [6]:
req = inference_pb2.InferenceRequest()
_ = text_format.Parse(config, req)

In [7]:
runner = inference.Runner()
runner.start(req)

canvas, alignment = runner.make_canvas(corner, subvol_size)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [8]:
relative_seed = (seed[0]-corner[0],seed[1]-corner[1],seed[2]-corner[2])


if INTERACTIVE:
    nb_iters = canvas.segment_at(relative_seed,
                      dynamic_image=inference.DynamicImage(),
                      vis_update_every=100, vis_fixed_z=False)
    print('nb_iters:' + str(nb_iters))
else:
    ## non interactive segmentation = much faster
    # (also, there's something wrong with the saving of the results in interactive mode)
    # The seed has to be specified as a point in 'seed_policy_args' (see the config above)
    canvas.segment_all(seed_policy=runner.get_seed_policy(corner, subvol_size))    
    seg_path = storage.segmentation_path(
        runner.request.segmentation_output_dir, corner)
    prob_path = storage.object_prob_path(
        runner.request.segmentation_output_dir, corner)
    
    runner.save_segmentation(canvas, alignment, seg_path, prob_path)

    
    print('number of voxels with nonzero probability estimates: ' + str(len(np.nonzero(canvas.seg_prob)[0])))

nb_iters:31360


In [9]:
if EXTRACT_ZIPS:
    resultdir =  os.path.join('/app/julien/validation/sumbul/single_image/Image' + current + '_01/segmentation_trial' + trial, str(corner[2]), str(corner[1]))

    path_to_zip1 = os.path.join(resultdir, 'seg-' + str(corner[2]) + '_' + str(corner[1]) + '_' + str(corner[0]) + '.prob')
    path_to_zip2 = os.path.join(resultdir, 'seg-' + str(corner[2]) + '_' + str(corner[1]) + '_' + str(corner[0]) + '.npz')


    zip_ref = zipfile.ZipFile(path_to_zip1, 'r')
    zip_ref.extractall(resultdir)
    zip_ref.close()

    zip_ref = zipfile.ZipFile(path_to_zip2, 'r')
    zip_ref.extractall(resultdir)
    zip_ref.close()